In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

import utilities_plot as up
import utilities_latent as ul
from utilities_base import VAE, load_data

In [ ]:
data, data_id_dict, child_id_dict, word_dict, category_dict = load_data(
    ["data", "data_id_dict", "child_id_dict", "word_dict", "category_dict"]
)
word_count = len(word_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE().to(device)
model.eval()
model.load_state_dict(torch.load("tmp/best_model.pth"))
# model.load_state_dict(torch.load("tmp/model_state_dict.pth"))

In [ ]:
# スプライン補完のための点群を指定

xy = [
    (2, 2.5),
    (-1, 2),
    (2, 1.5),
    (-2, 1),
    (2, 0.5),
    (-3, 0),
    (2, -0.5),
    (-2, -1),
    (2, -1.5),
    (-1, -2),
    (2, -2.5),
    (5, 0),
    (2, 2.5),
]

n = 200
Zs = ul.spline_interpolation(xy, n)

In [ ]:
# どのように補完されたかをプロット

figs = {}
figs["tmp"] = plt.subplots()
fig, ax = figs["tmp"]
up.plot_x(model, data, ax)
ax.plot(Zs[:, 0], Zs[:, 1], color="tab:orange", linestyle="--")
ax.legend()

In [ ]:
# ファイル名の設定
tmp_image_folder = Path("images/tmp")
tmp_expectation_image_name = "expectation"
tmp_point_image_name = "point"
output_gif_path = Path("images/gif/expectation.gif")

# gifにするファイルを選択
image_expectation_dirs1 = [
    str(tmp_image_folder / tmp_expectation_image_name) + f"_{i}.png" for i in range(n)
]
image_expectation_dirs2 = [
    str(tmp_image_folder / tmp_point_image_name) + f"_{i}.png" for i in range(n)
]

In [ ]:
# 点Zsにおけるのカテゴリーごとの期待値を計算しプロット
# そのときのZsの位置もプロット
# gifファイル作成に備えて保存

ul.save_expectation_plot_with_category(
    model, Zs, tmp_image_folder, tmp_expectation_image_name
)
ul.save_zs_plot(model, Zs, tmp_image_folder, tmp_point_image_name)

In [ ]:
# gifにして保存
ul.make_combined_gif(image_expectation_dirs1, image_expectation_dirs2, output_gif_path)

In [ ]:
# スプライン補完のための点群を指定

xy1 = [(0, 2), (-2, 0), (0, -2), (-3, 0), (0, 2)]
xy2 = [(2, 1),(1, 0), (2, -1), (2, 0), (2, 1)]

n = 50
Zs1 = ul.spline_interpolation(xy1, n)
Zs2 = ul.spline_interpolation(xy2, n)

In [ ]:
# どのように補完されたかをプロット

figs = {}
figs["tmp"] = plt.subplots()
fig, ax = figs["tmp"]
xy1 = np.array(xy1)
xy2 = np.array(xy2)
up.plot_x(model, data, ax)
ax.plot(Zs1[:, 0], Zs1[:, 1], color="tab:orange", linestyle="--", label="spline1")
ax.plot(Zs2[:, 0], Zs2[:, 1], color="tab:green", linestyle="--", label="spline2")
ax.scatter(xy1[:, 0], xy1[:, 1], color="tab:orange", label="spline refs1")
ax.scatter(xy2[:, 0], xy2[:, 1], color="tab:green", label="spline refs2")
ax.legend()

In [ ]:
# ファイル名の設定
tmp_image_folder = Path("images/tmp")
tmp_expectation_diff_image_name = "expectation_diff"
tmp_point_diff_image_name = "point_diff"
output_gif_diff_path = Path("images/gif/diff.gif")

# gifにするファイルを選択
image_expectation_diff_dirs1 = [
    str(tmp_image_folder / tmp_expectation_diff_image_name) + f"_{i}.png" for i in range(n)
]
image_expectation_diff_dirs2 = [
    str(tmp_image_folder / tmp_point_diff_image_name) + f"_{i}.png" for i in range(n)
]

In [ ]:
# 点Zsにおけるのカテゴリーごとの期待値を計算しプロット
# そのときのZsの位置もプロット
# gifファイル作成に備えて保存

ul.save_expectation_diff_plot_with_category(
    model, Zs1, Zs2, "A", "B", tmp_image_folder, tmp_expectation_diff_image_name
)
ul.save_zs_diff_plot(model, Zs1, Zs2, "A", "B", tmp_image_folder, tmp_point_diff_image_name)

In [ ]:
# gifにして保存
ul.make_combined_gif(
    image_expectation_diff_dirs1, image_expectation_diff_dirs2, output_gif_diff_path
)

In [ ]:
# アルファベットとカテゴリーの対応関係
df_category = pd.DataFrame(
    {"category": list(category_dict.keys())}, index=[chr(65 + i) for i in range(22)]
)
df_category